In [7]:
from os import listdir
import os

#apparently doesn't accept wildcards
# print(listdir())


hundred = 100
current_page = 0
for i in range(0, 7):
    if os.path.isfile(f"system_list_{hundred}.pickle") is True:
        current_page = hundred
    print(current_page)
    # print(os.path.isfile(f"system_list_{hundred}.pickle"))
    hundred += 100

['.git', '.gitignore', '.idea', '.ruff_cache', 'automate_mining_scrap.ipynb', 'basic_navigation.py', 'client.py', 'crud.py', 'current_systems_jumpgate_map.py', 'data_decode.py', 'data_to_sql.py', 'df_print_full.py', 'df_to_sql.py', 'get_player_credits.py', 'get_ships.py', 'get_ships_list.py', 'handle_status_codes.py', 'july18th_system_list.pickle', 'july23_reset_backup.sql', 'jumpgate_lookup.py', 'jumpgate_waypoint_query.sql', 'jump_gate_mapping.ipynb', 'jump_gate_navigation.py', 'key.py', 'log_market_transaction.py', 'log_status_code.py', 'market_data_query.sql', 'market_prices_sql.sql', 'market_refresher.py', 'market_transactions_sql.sql', 'MiningSurvey.py', 'mining_ships.py', 'mining_survey_ranking.py', 'mock_surveyresult.py', 'models.py', 'navigation_tinkering.ipynb', 'OLDspace_traders_api_client', 'OLDsystem_list.pickle', 'orm_stuff', 'peewee_models.py', 'peewee_models_tinkering.ipynb', 'player.py', 'plot_a_course.py', 'reworking_sql_interaction.ipynb', 'run_on_server_reset.py', '

In [15]:
import pickle


hundred = 100
current_page = 0

merged_list = []
for i in range(0, 7):
    if os.path.isfile(f"system_list_{hundred}.pickle") is True:
        with open(f"system_list_{hundred}.pickle", "rb") as f:
            merged_list.append(pickle.load(f))
    # print(os.path.isfile(f"system_list_{hundred}.pickle"))
    hundred += 100
print(len(merged_list[1]))

# print(merged_list[1])


3980


In [6]:
import os
import pickle
from client import client
from space_traders_api_client.api.systems import (
    get_systems,
)
from data_decode import data_decode
from time import sleep


def _depaginate_results(paginated_system_list: list[dict]) -> list[dict]:
    """

    just a cleaning step in _get_all_systems

    :param paginated_system_list: this is the paginated list returned by get_all_systems
    :return: just appending each system in each page of systems into a new unpaginated list
    """
    system_list = []

    for i in range(0, len(paginated_system_list) - 1):
        for system in paginated_system_list[i]:
            system_list.append(system)

    return system_list




def _get_all_systems(page: int = 1, limit: int = 20, client=client, save_increment:int=50):
    """
    todo this seems ratelimited by plane wireless timing out


    this returns a list of all of the systems from get_systems
    there are 12k systems, pagination max is 20 per page, so 600 pages
    :param page: int 1-600
    :param limit: 20 max
    :param client: defined in shiplist

    :return:
    """
    full_systems_list = []
    # probably should be more granular than 100
    page = 0
    # page_save_increment = save_increment
    page_hundred = 100

    #todo this actually requires more logic than i have on me right now, so i'll just pretend the arg doesn't exist
    for i in range(0, 650, 50):
    # for i in range(50, 150, 50):
        if os.path.isfile(f"system_list_{i}.pickle") is True:
            page = i
    
    
    # partial_system_list = []
    while page < 600:
        # i can't think through the math right now, so i'll just sleep every two pages
        if page % 2 == 0:
            sleep(0.5)
        get_systems_response = get_systems.sync_detailed(page=page, limit=20, client=client)
        #todo i really need to handle status codes generally
        if get_systems_response.status_code in [200, 201]:
            print(f"page {page} of 600, {page / 600}")
            pass
        else:
            print(get_systems_response)

        full_systems_list.append(data_decode(get_systems_response.content))
        if page % 50 == 0:
            """
            this is jenky 
            """
            #fix names 
            cleaned_hundred_pages = _depaginate_results(full_systems_list)
            with open(f"system_list_{page}.pickle", "wb") as f:
                pickle.dump(cleaned_hundred_pages, f)
                # page_hundred += 100

        page += 1
    cleaned_systems_list = _depaginate_results(full_systems_list)

    return cleaned_systems_list


system_list_test = _get_all_systems()




IndentationError: expected an indented block after 'for' statement on line 30 (1675756870.py, line 32)

In [ ]:
#there's probably some room to do this as a distributed task either via multiprocessing or async within the constraints of the rate limit

In [30]:
"""

THIS ONE -- still needs some work, but it's most of the way there



"""


import os
import pickle
from client import client
from space_traders_api_client.api.systems import (
    get_systems,
)
from data_decode import data_decode
from time import sleep
import math 

def _depaginate_results(paginated_system_list: list[dict]) -> list[dict]:
    """

    just a cleaning step in _get_all_systems

    :param paginated_system_list: this is the paginated list returned by get_all_systems
    :return: just appending each system in each page of systems into a new unpaginated list
    """
    system_list = []

    for i in range(0, len(paginated_system_list) - 1):
        for system in paginated_system_list[i]:
            system_list.append(system)

    return system_list

def _depaginate_dict_results(paginated_system_dict: dict) -> list:
    """
    might be trash
    
    :param paginated_system_dict: 
    :return: 
    """
    system_list = []
    for system in paginated_system_dict.values():
        print(system)
        system_list.append(system)

def get_all_systems_finer_grain():
    """
    this doesn't really resume-- it's another page vs id problem 
    
    
    :return: 
    """
    page = 1 
    # systems_list = []
    systems_dict = {}
    if os.path.isfile("in_process_systems_dict.pickle") is True:
        #this should depickle and set page to last key from dict 
        with open("in_process_systems_dict.pickle", "rb") as f:
            systems_dict = pickle.load(f)
        page = list(systems_dict.keys())[-1]
    
    elif os.path.isfile("in_process_systems_dict.pickle") is False:
        with open(f"in_process_systems_dict.pickle", "wb") as f:
            pickle.dump(systems_dict, f)
            
    while page < 601:
        # with open("in_process_systems_list.pickle", "rb") as f:
        #     systems_list = pickle.load(f)        
        with open("in_process_systems_dict.pickle", "rb") as f:
            systems_dict = pickle.load(f)
        if page % 2 == 0:
            sleep(0.5)
        get_systems_response = get_systems.sync_detailed(page=page, limit=20, client=client)
        print(get_systems_response)
        #todo i really need to handle status codes generally
        if get_systems_response.status_code in [200, 201]:
            #todo lol i'm too tired to round this 
            print(f"page {page} of 600, {str((page / 600)*100)[0:6]}%")
            pass
        else:
            print(get_systems_response)
        # depaginated_list = []

        # depaginated_list.append(_depaginate_results(systems_list))
        # systems_list.append(_depaginate_results(data_decode(get_systems_response.content)))
        systems_dict[page] = data_decode(get_systems_response.content)
        print(systems_dict[600])
        #todo 
        # with open(f"in_process_systems_list.pickle", "wb") as f:
        #     pickle.dump(systems_list, f)        
    
        with open(f"in_process_systems_dict.pickle", "wb") as f:
            pickle.dump(systems_dict, f)
        page+=1
    systems_list = _depaginate_results(list(systems_dict.values))
    return systems_list

systems_dict = get_all_systems_finer_grain()


#todo there might be a new step to get this out of a dict 


Response(status_code=<HTTPStatus.OK: 200>, content=b'{"data":[{"symbol":"X1-ZZ20","sectorSymbol":"X1","type":"ORANGE_STAR","x":-29348,"y":44582,"waypoints":[{"symbol":"X1-ZZ20-70000E","type":"PLANET","x":-31,"y":21},{"symbol":"X1-ZZ20-43841B","type":"MOON","x":-31,"y":21},{"symbol":"X1-ZZ20-40882Z","type":"MOON","x":-31,"y":21},{"symbol":"X1-ZZ20-15603F","type":"PLANET","x":-45,"y":7},{"symbol":"X1-ZZ20-70584D","type":"MOON","x":-45,"y":7},{"symbol":"X1-ZZ20-51645B","type":"GAS_GIANT","x":-6,"y":66},{"symbol":"X1-ZZ20-18286B","type":"ASTEROID_FIELD","x":-62,"y":-49}],"factions":[]},{"symbol":"X1-NP68","sectorSymbol":"X1","type":"RED_STAR","x":-23673,"y":51576,"waypoints":[{"symbol":"X1-NP68-94250A","type":"PLANET","x":-39,"y":15},{"symbol":"X1-NP68-32331F","type":"MOON","x":-39,"y":15},{"symbol":"X1-NP68-10912A","type":"PLANET","x":61,"y":-11},{"symbol":"X1-NP68-03173F","type":"MOON","x":61,"y":-11},{"symbol":"X1-NP68-37954F","type":"PLANET","x":-67,"y":55},{"symbol":"X1-NP68-50755X","

TypeError: 'builtin_function_or_method' object is not iterable

In [28]:
get_systems_response = get_systems.sync_detailed(page=600, limit=20, client=client)
# print(data_decode(get_systems_response.content))
uhh = data_decode(get_systems_response.content)
system_list = []

for system in uhh:
    print(system)


# for k, v in enumerate(systems_dict.items()):
#     print(k, v)

# for system in uhh: 
#     # print(system.values())
#     system_list.append(list(system.values()))
#     
# print(system_list)
# print(_depaginate_results(uhh))

# #todo i really need to handle status codes generally
# if get_systems_response.status_code in [200, 201]:
#     #todo lol i'm too tired to round this 
#     # print(f"page {page} of 600, {str((page / 600)*100)[0:6]}%")
#     pass
# else:
    # print(get_systems_response)
# depaginated_list = []

# depaginated_list.append(_depaginate_results(systems_list))
# systems_list.append(_depaginate_results(data_decode(get_systems_response.content)))
# systems_dict[600] = _depaginate_results(data_decode(get_systems_response.content))
# print(_depaginate_results(data_decode(get_systems_response.content)))
# print(systems_dict[600])

{'symbol': 'X1-ZZ20', 'sectorSymbol': 'X1', 'type': 'ORANGE_STAR', 'x': -29348, 'y': 44582, 'waypoints': [{'symbol': 'X1-ZZ20-70000E', 'type': 'PLANET', 'x': -31, 'y': 21}, {'symbol': 'X1-ZZ20-43841B', 'type': 'MOON', 'x': -31, 'y': 21}, {'symbol': 'X1-ZZ20-40882Z', 'type': 'MOON', 'x': -31, 'y': 21}, {'symbol': 'X1-ZZ20-15603F', 'type': 'PLANET', 'x': -45, 'y': 7}, {'symbol': 'X1-ZZ20-70584D', 'type': 'MOON', 'x': -45, 'y': 7}, {'symbol': 'X1-ZZ20-51645B', 'type': 'GAS_GIANT', 'x': -6, 'y': 66}, {'symbol': 'X1-ZZ20-18286B', 'type': 'ASTEROID_FIELD', 'x': -62, 'y': -49}], 'factions': []}
{'symbol': 'X1-NP68', 'sectorSymbol': 'X1', 'type': 'RED_STAR', 'x': -23673, 'y': 51576, 'waypoints': [{'symbol': 'X1-NP68-94250A', 'type': 'PLANET', 'x': -39, 'y': 15}, {'symbol': 'X1-NP68-32331F', 'type': 'MOON', 'x': -39, 'y': 15}, {'symbol': 'X1-NP68-10912A', 'type': 'PLANET', 'x': 61, 'y': -11}, {'symbol': 'X1-NP68-03173F', 'type': 'MOON', 'x': 61, 'y': -11}, {'symbol': 'X1-NP68-37954F', 'type': '

In [9]:


blah_dict = {}
for i in range(0, 20):
    blah_dict[i] = f'my entire ass {i}'

print(list(blah_dict.keys())[-1])



19


In [ ]:
"""
should be able to use postgresql instead of the intermediary pickle, but it will need another api call per system, so it's not really
a solution to the current timeout problem unless it sleeps about a second per system-- actually the pagination means that you can't lookup 
the system by id 



"""
import os
import pickle
from client import client
from space_traders_api_client.api.systems import (
    get_systems,
)
from data_decode import data_decode
from time import sleep

def _depaginate_results(paginated_system_list: list[dict]) -> list[dict]:
    """

    just a cleaning step in _get_all_systems

    :param paginated_system_list: this is the paginated list returned by get_all_systems
    :return: just appending each system in each page of systems into a new unpaginated list
    """
    system_list = []

    for i in range(0, len(paginated_system_list) - 1):
        for system in paginated_system_list[i]:
            system_list.append(system)

    return system_list

def get_systems_to_sql()



def system_waypoints_to_sql(system_symbol, waypoint_symbol, waypoint_type, jumpgate_list):













In [17]:
for i in range(0, 650, 50):
    print(i)
    

0
50
100
150
200
250
300
350
400
450
500
550
600


In [ ]:

"""
you're very tired, this might not be the best thing to work on
"""











